**The training and benchmark was ran in Kaggle Notebook with logging to WandB. The logging codes and plotting codes are commented out here to prevent error while running in different environment and to shorten notebook run time. Besides max_epoch, Kaggle notebook maximum duration of 12 hours is also another training stopper**

# 1. Install libraries

In [ ]:
!pip install snntorch --quiet

In [ ]:
!pip show snntorch

Name: snntorch
Version: 0.9.1
Summary: Deep learning with spiking neural networks.
Home-page: https://github.com/jeshraghian/snntorch
Author: Jason K. Eshraghian
Author-email: jeshragh@ucsc.edu
License: MIT License
Location: /opt/conda/lib/python3.10/site-packages
Requires: matplotlib, nir, nirtorch, numpy, pandas, torch
Required-by: 


In [ ]:
!pip install --quiet lightning

In [ ]:
!pip show lightning

Name: lightning
Version: 2.3.3
Summary: The Deep Learning framework to train, deploy, and ship AI products Lightning fast.
Home-page: https://github.com/Lightning-AI/lightning
Author: Lightning AI et al.
Author-email: pytorch@lightning.ai
License: Apache-2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: fsspec, lightning-utilities, numpy, packaging, pytorch-lightning, PyYAML, torch, torchmetrics, tqdm, typing-extensions
Required-by: 


In [ ]:
# !pip install --quiet wandb # logging code

In [ ]:
# !pip show wandb # logging code

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import snntorch as snn
from snntorch import surrogate

import torch
from torch.utils.data import DataLoader, Subset

import tqdm

import matplotlib.pyplot as plt

import lightning as L
import torch.nn.functional as F
from torchmetrics.functional import r2_score
from torchmetrics.functional import pearson_corrcoef

# import wandb # logging code
# from kaggle_secrets import UserSecretsClient # logging code

from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import Callback

from lightning.pytorch import seed_everything

from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import time
import random

In [ ]:
# setup
seed_num = 42 # seed used: 42, 26, 0, 1, 1234
seed_everything(seed_num, workers=True)
torch.manual_seed(seed_num)
random.seed(seed_num)
np.random.seed(seed_num)

# dataset
data_dir = './nonhuman-primate-reaching/'
filename = 'indy_20160622_01'
bin_width = 0.004
batch_size = 1024
num_workers = 3
remove_segments_inactive = False

def seed_worker(worker_id):
    np.random.seed(seed_num)
    random.seed(seed_num)

g = torch.Generator()
g.manual_seed(seed_num)

# model
dropout = 0.3
output_dims = [32, 24, 0]
tb_array=True
reset_mechanism='zero'
lr = 0.0010
optim ="AdamW"
weight_decay = 0.0127
T_max = 90
scheduler = True

# training
accelerator = 'cpu'
devices = '1'
strategy = L.pytorch.strategies.SingleDeviceStrategy()
sync_dist = (accelerator=='gpu')
min_epoch = None
max_epoch = 90
patience = 10

# # logging code
# user_secrets = UserSecretsClient()
# wandb_api = user_secrets.get_secret("wandb_api")
# wandb.login(key=wandb_api)
# run_name = f"sMLP2, {seed_num}, {filename}"

INFO: Seed set to 42


In [ ]:
!pip install neurobench --quiet

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
featuretools 1.30.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.2 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shape

In [ ]:
!pip show neurobench

Name: neurobench
Version: 1.0.6
Summary: Collaborative, Fair, and Representative Benchmarks for Neuromorphic Computing
Home-page: 
Author: NeuroBench Team
Author-email: neurobench@googlegroups.com
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: llvmlite, numba, numpy, snntorch, tonic, torch, torchaudio, tqdm
Required-by: 


In [ ]:
from neurobench.datasets import PrimateReaching
from neurobench.models.torch_model import TorchModel
from neurobench.benchmarks import Benchmark

# 2. Prepare dataset

In [ ]:
class NHPDataModule(L.LightningDataModule):
    def __init__(self, data_dir, filename, bin_width, remove_segments_inactive, batch_size, num_workers, seed_worker, generator):
        super().__init__()
        self.data_dir = data_dir
        self.filename = filename
        self.bin_width = bin_width
        self.remove_segments_inactive = remove_segments_inactive
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.seed_worker = seed_worker
        self.generator = generator

    def prepare_data(self):
        # download
        PrimateReaching(file_path=self.data_dir, filename=self.filename,
                        num_steps=1, train_ratio=0.5, bin_width=self.bin_width,
                        biological_delay=0, remove_segments_inactive=self.remove_segments_inactive, download=False)

    def setup(self, stage=None):
        dataset = PrimateReaching(file_path=self.data_dir, filename=self.filename,
                        num_steps=1, train_ratio=0.5, bin_width=self.bin_width,
                        biological_delay=0, remove_segments_inactive=self.remove_segments_inactive, download=False)

        # Handle Loco's recording
        dataset.samples = dataset.samples[0:96,:]

        # Assign train/val datasets for use in dataloaders
        self.nhp_train = Subset(dataset, dataset.ind_train)
        self.nhp_val = Subset(dataset, dataset.ind_val)

        # Assign test dataset for use in dataloader(s)
        self.nhp_test = Subset(dataset, dataset.ind_test)

    def train_dataloader(self):
        return DataLoader(self.nhp_train, batch_size=self.batch_size, shuffle=False, num_workers = self.num_workers, worker_init_fn=self.seed_worker, generator=self.generator)

    def val_dataloader(self):
        return DataLoader(self.nhp_val, batch_size=len(self.nhp_val), shuffle=False, num_workers = self.num_workers, worker_init_fn=self.seed_worker, generator=self.generator)

    def test_dataloader(self):
        return DataLoader(self.nhp_test, batch_size=len(self.nhp_test), shuffle=False, num_workers = self.num_workers, worker_init_fn=self.seed_worker, generator=self.generator)

In [ ]:
dm = NHPDataModule(data_dir=data_dir,
                   filename=filename,
                   bin_width=bin_width,
                   remove_segments_inactive=remove_segments_inactive,
                   batch_size=batch_size,
                   num_workers=num_workers,
                   seed_worker=seed_worker,
                   generator=g)

# 3. Construct model

In [ ]:
from torch import Tensor
from torch.nn.parameter import Parameter
from torch.nn import functional as F, init
import math

In [ ]:
class Scale2to2(nn.Module):
    weight: Tensor

    def __init__(
        self,
        bias: bool = True,
        device=None,
        dtype=None,
    ) -> None:
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.weight = Parameter(
            torch.empty((1, 2), **factory_kwargs)
        )
        self.register_parameter("bias", None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(in_features), 1/sqrt(in_features)). For details, see
        # https://github.com/pytorch/pytorch/issues/57109
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return torch.mul(input, self.weight)

    def extra_repr(self) -> str:
        return f"in_features={2}, out_features={2}, bias={self.bias is not None}"

In [ ]:
class Net(nn.Module):
    def __init__(self, dropout, output_dims, tb_array, reset_mechanism, input_dim=96):
        super().__init__()

        self.input_dim = input_dim
        self.final_output_dim = 2
        self.surrogate = surrogate.atan(alpha=2)

        self.layers = []

        for output_dim in output_dims:
            if (output_dim!=0):
                layer = {}
                layer['dropout'] = nn.Dropout(dropout)
                layer['fc'] = nn.Linear(self.input_dim, output_dim)

                if (tb_array):
                    layer['beta'] = torch.rand((output_dim), dtype=torch.float)
                    layer['threshold'] = torch.rand((output_dim), dtype=torch.float)
                    layer['lif'] = snn.Leaky(beta=layer['beta'], spike_grad=self.surrogate, threshold=layer['threshold'],
                                            learn_beta=True, learn_threshold=True, reset_mechanism=reset_mechanism)
                else:
                    layer['lif'] = snn.Leaky(beta=0.96, spike_grad=self.surrogate,
                                            learn_beta=True, learn_threshold=True, reset_mechanism=reset_mechanism)
                self.input_dim = output_dim

                # Add the layers to the internal representation
                self.add_module(f'dropout{len(self.layers)}', layer['dropout'])
                self.add_module(f'fc{len(self.layers)}', layer['fc'])
                self.add_module(f'lif{len(self.layers)}', layer['lif'])

                # Add the layers to our layer list.
                self.layers.append(layer)

        layer = {}
        layer['fc'] = nn.Linear(self.input_dim, self.final_output_dim)
        if (tb_array):
            layer['beta'] = torch.rand((self.final_output_dim), dtype=torch.float)
            layer['threshold'] = torch.rand((self.final_output_dim), dtype=torch.float)
            layer['lif'] = snn.Leaky(beta=layer['beta'], spike_grad=self.surrogate, threshold=layer['threshold'],
                                     learn_beta=True, learn_threshold=True, reset_mechanism='none')
        else:
            layer['lif'] = snn.Leaky(beta=0.96, spike_grad=self.surrogate,
                                     learn_beta=True, learn_threshold=True, reset_mechanism='none')
        layer['scale'] = Scale2to2()

        # Add the layers to the internal representation
        self.add_module(f'fc{len(self.layers)}', layer['fc'])
        self.add_module(f'lif{len(self.layers)}', layer['lif'])
        self.add_module(f'scale{len(self.layers)}', layer['scale'])

        # Add the layers to our layer list.
        self.layers.append(layer)

    def reset(self):
        # Initialize all the neurons in all layers
        for layer in self.layers:
            layer['mem'] = layer['lif'].init_leaky()

    def forward(self, x):
        # here x is expected to be shape (len_series, 1, input_dim)

        self.reset()

        predictions = []

        for step in range(x.shape[0]):
#             print('idx')
#             print(step)
#             print()
            cur = x[step, ...].squeeze()
#             print('x')
#             print(cur)
#             print()

            for index, layer in enumerate(self.layers):
                if (index==len(self.layers)-1):
                    cur = layer['fc'](cur)
#                     print('cur3')
#                     print(cur)
#                     print()
                    cur, layer['mem'] = layer['lif'](cur, layer['mem'])
#                     print('mem_out')
#                     print(layer['mem'])
#                     print()

                    pred = layer['scale'](layer['mem'])
#                     print('fc_scale')
#                     print(pred)
#                     print('====================================================================')
#                     print()


                else:
                    cur = layer['fc'](cur)
#                     print('fc'+str(index+1))
#                     print(cur)
#                     print()
                    cur = layer['dropout'](cur)
#                     print('do'+str(index+1))
#                     print(cur)
#                     print()
                    cur, layer['mem'] = layer['lif'](cur, layer['mem'])
#                     print('spk'+str(index+1))
#                     print(cur)
#                     print()

#             if (step==1):
#                 break

            predictions.append(pred)#self.layers[-1]['mem'])

        return torch.stack(predictions).squeeze()

In [ ]:
class LitModel(L.LightningModule):
    def __init__(self, dropout, output_dims, tb_array, reset_mechanism, lr, optim, weight_decay, T_max, scheduler, input_dim=96):
        super().__init__()
        self.model = Net(dropout, output_dims, tb_array, reset_mechanism)
        self.lr = lr
        self.optim = optim
        self.weight_decay = weight_decay
        self.scheduler = scheduler
        self.T_max = T_max

    def forward(self, feature):
        return self.model(feature)

    def configure_optimizers(self):
        if (self.optim=="Adam"):
            optimizer = torch.optim.Adam(params=self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        elif (self.optim=="RMSprop"):
            optimizer = torch.optim.RMSprop(params=self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        elif (self.optim=="AdamW"):
            optimizer = torch.optim.AdamW(params=self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        if (self.scheduler):
            lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, self.T_max)
            return [optimizer], [lr_scheduler]
        else:
            return [optimizer]

    def training_step(self, train_batch, batch_idx):
        _, loss, r2score, cc = self._get_preds_loss_accuracy(train_batch, batch_idx)

        self.log('train_loss', loss, prog_bar=True, on_epoch=True)
        self.log('train_r2score',r2score, prog_bar=True, on_epoch=True)
        self.log('train_cc',cc, prog_bar=True, on_epoch=True)

        return loss

    def validation_step(self, val_batch, batch_idx):
        mem, loss, r2score, cc = self._get_preds_loss_accuracy(val_batch, batch_idx)

        self.log('val_loss',loss, prog_bar=True)
        self.log('val_r2score',r2score, prog_bar=True)
        self.log('val_cc',cc, prog_bar=True)

        return mem

    def _get_preds_loss_accuracy(self, batch, batch_idx):
        '''convenience function since train/valid/test steps are similar'''
        feature, label = batch
        mem = self(feature)
        loss = F.mse_loss(mem, label) # calculate loss
        r2 = r2_score(mem, label, multioutput='uniform_average')
        cc = (sum(pearson_corrcoef(mem, label))/2).item()

        return mem, loss, r2, cc

In [ ]:
model = LitModel(dropout=dropout,
                 output_dims=output_dims,
                 tb_array=tb_array,
                 reset_mechanism=reset_mechanism,
                 lr=lr,
                 optim=optim,
                 weight_decay=weight_decay,
                 T_max=T_max,
                 scheduler=scheduler)

In [ ]:
print(model)

LitModel(
  (model): Net(
    (dropout0): Dropout(p=0.3, inplace=False)
    (fc0): Linear(in_features=96, out_features=32, bias=True)
    (lif0): Leaky()
    (dropout1): Dropout(p=0.3, inplace=False)
    (fc1): Linear(in_features=32, out_features=24, bias=True)
    (lif1): Leaky()
    (fc2): Linear(in_features=24, out_features=2, bias=True)
    (lif2): Leaky()
    (scale2): Scale2to2(in_features=2, out_features=2, bias=False)
  )
)


In [ ]:
# model = LitModel(dropout=dropout,
#                  output_dims=output_dims,
#                  tb_array=tb_array,
#                  reset_mechanism=reset_mechanism,
#                  lr=lr,
#                  optim=optim,
#                  weight_decay=weight_decay,
#                  T_max=T_max,
#                  scheduler=scheduler)

# dm.setup()
# # model.eval()
# # with torch.no_grad():
# i=0
# for samples, labels in iter(dm.train_dataloader()):
#     print(i)
#     mem = model(samples)
#     if (i==0):
#         break
#     i+=1
#     print()

# 4. Construct training loop

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath='./model-checkpoint/', monitor='val_r2score', mode='max')

# wandb_logger = WandbLogger(project='grandChallenge', name=run_name, log_model='all') # logging code

In [ ]:
trainer=L.Trainer(
    accelerator=accelerator,
    devices=devices,
    strategy=strategy,
#     logger=wandb_logger, # logging code
    callbacks=[checkpoint_callback, EarlyStopping(monitor="val_loss", min_delta=0, patience=patience, mode='min', verbose=True)],
    min_epochs=min_epoch,
    max_epochs=max_epoch,
    deterministic=True)

INFO: GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


In [ ]:
start_time = time.time()
trainer.fit(model, dm)
end_time = time.time()

Loading indy_20160622_01.mat


2024-08-02 05:23:47.782984: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 05:23:47.783144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 05:23:47.991584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading indy_20160622_01.mat


/opt/conda/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /kaggle/working exists and is not empty.
INFO: 
  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | Net  | 4.1 K  | train
---------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.016     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Monitored metric val_loss did not improve in the last 10 records. Best score: 0.022. Signaling Trainer to stop.


In [ ]:
print(f"Time taken: {end_time - start_time} seconds\n")

Time taken: 20369.600644350052 seconds



# 5. Benchmark

In [ ]:
if (remove_segments_inactive):
    remove_segments_inactive=False
    dm = NHPDataModule(data_dir=data_dir,
                       filename=filename,
                       bin_width=bin_width,
                       remove_segments_inactive=remove_segments_inactive,
                       batch_size=batch_size,
                       num_workers=num_workers,
                       seed_worker=seed_worker,
                       generator=g)
    dm.setup()

In [ ]:
checkpoint_callback.best_model_path

'/kaggle/working/epoch=21-step=8030.ckpt'

In [ ]:
# # plotting code
# model = LitModel.load_from_checkpoint(checkpoint_callback.best_model_path,
#                                       dropout=dropout,
#                                       output_dims=output_dims,
#                                       tb_array=tb_array,
#                                       reset_mechanism=reset_mechanism,
#                                       lr=lr,
#                                       optim=optim,
#                                       weight_decay=weight_decay,
#                                       T_max=T_max,
#                                       scheduler=scheduler)
# model.eval()

In [ ]:
# # plotting code
# with torch.no_grad():
#     test_batch = iter(dm.test_dataloader())
#     for feature, label in test_batch:
#         mem = model(feature)

#         cc_test = sum(pearson_corrcoef(mem, label))/2

#     fig, ax = plt.subplots()
#     ax.plot(mem[:,0].detach().numpy(), label="Output")
#     ax.plot(label[:,0], '--', label="Target")
#     ax.set_xlabel("Time")
#     ax.set_ylabel("mm/s")
#     ax.legend(loc='best')

# #     wandb.log({"Testing X-axis Output": fig}) # logging code
#     fig.show()

#     fig, ax = plt.subplots()
#     ax.plot(mem[:,1].detach().numpy(), label="Output")
#     ax.plot(label[:,1], '--', label="Target")
#     ax.set_xlabel("Time")
#     ax.set_ylabel("mm/s")
#     ax.legend(loc='best')

# #     wandb.log({"Testing Y-axis Output": fig}) # logging code
#     fig.show()

In [ ]:
model = LitModel.load_from_checkpoint(checkpoint_callback.best_model_path,
                                      dropout=dropout,
                                      output_dims=output_dims,
                                      tb_array=tb_array,
                                      reset_mechanism=reset_mechanism,
                                      lr=lr,
                                      optim=optim,
                                      weight_decay=weight_decay,
                                      T_max=T_max,
                                      scheduler=scheduler)
model.eval()

model = TorchModel(model) # using TorchModel instead of SNNTorchModel because the SNN iterates over dimension 0
model.add_activation_module(snn.SpikingNeuron)

In [ ]:
static_metrics = ["footprint"]
workload_metrics = ["r2", "synaptic_operations"]

# Benchmark expects the following:
benchmark = Benchmark(model, dm.test_dataloader(), [], [], [static_metrics, workload_metrics])
results = benchmark.run()
print(results)

# # logging code
# columns = list(results.keys())
# data = list(results.values())
# synaptic_operations = data[-1]
# columns.pop()
# columns += list(synaptic_operations.keys())
# columns += ["cc"]
# columns += ["seed"]
# data.pop()
# data += list(synaptic_operations.values())
# data += [cc_test.item()]
# data += [seed_num]

# wandb_logger.log_table(key='benchmark_table', columns=columns, data=[data]) # logging code

Running benchmark


100%|██████████| 1/1 [07:36<00:00, 456.83s/it]


{'footprint': 16292, 'r2': 0.6780295968055725, 'synaptic_operations': {'Effective_MACs': 0.0, 'Effective_ACs': 429.5975893630645, 'Dense': 3888.0}}


In [ ]:
# SMLP2
# seed used: 42, 26, 0, 1, 1234

# recording order: [indy_20160622_01, indy_20160630_01, indy_20170131_02, loco_20170210_03, loco_20170215_02, loco_20170301_05]

# array of mean of 5 seeds for each recording, followed by mean of 6 recordings at the end of the array
# # CC: [0.817733454704285, 0.731160259246826, 0.747678899765015, 0.524604910612106, 0.489216667413712, 0.586902409791946] 0.649549433588982
# # R2: [0.658298170566559, 0.51222767829895, 0.516845268011093, 0.248315578699112, 0.178188401460648, 0.341011899709702] 0.409147832791011
# # Footprint: [16292, 16292, 16292, 16292, 16292, 16292] 16292
# # Dense: [3888, 3888, 3888, 3888, 3888, 3888] 3888
# # MACs: [0, 0, 0, 0, 0, 0] 0
# # ACs: [415.3644837847, 281.040002233722, 230.640333103078, 194.392273074267, 174.197703071717, 233.91684677391] 254.925273673565

# array of standard error of 5 seeds for each recording with order: indy_20160622_01, indy_20160630_01, indy_20170131_02, loco_20170210_03, loco_20170215_02, loco_20170301_05
# # CC: [0.0123900046833928, 0.00702905573566957, 0.0294282207956668, 0.0362570257348538, 0.066428503592395, 0.113197790193477]
# # R2: [0.0186805233086215, 0.012501484788652, 0.0393299506748004, 0.0376720858040179, 0.0427851897974436, 0.119494191977163]
# # Footprint: [0, 0, 0, 0, 0, 0]
# # Dense: [0, 0, 0, 0, 0, 0]
# # MACs: [0, 0, 0, 0, 0, 0]
# # ACs: [11.7367280721293, 13.4837011040563, 13.4582934615064, 17.9449377143127, 13.7148165792977, 12.9328506900957]

In [ ]:
# wandb.finish() # logging code